In [0]:
#needed libraries 

import numpy as np
import matplotlib.pyplot as plt
import random as rand
import time 

In [0]:
#Global variables 

#environment robot lives in is L by H 
L = 5 #max size of x
H = 6 #max size of y

p_e = 0.01 
gamma = 0.9 

#class states defines the robot state 
#the robot takes in an x,y coordinate for its position 
#this was made into an object in order to create a map of state to action
class state: 
  def __init__(self, x, y): 
    self.x = x
    self.y = y

#class action defines the robot's action 
#it contains an integer for move 
#global definitions for all moves (to avoid confusion)
left = 0 
right = 1 
up = 2 
down = 3 
none = 4
obstacle = 5
class action: 
  def __init__(self, move):
    self.move = move
  #move can be: left, right, up, down, none

1(c) / 2(a): Function that will return probability of transition from one state to the next state given an action.  This was originally made to not include obstalces but has since been modified to include them. 

In [0]:
def get_probability (s, a, n_s): 

    # probability of staying still if choosing to not move is 1
    if (a == none):
      if s.x == n_s.x and s.y == n_s.y:
        return 1
      else:
        return 0 

    # probability of moving to intended direction that is within bounds is 1 - Pe + Pe/4
    # probability of moving to unintended direction that is within bounds is Pe - Pe/4
    elif ( s.x - 1 == n_s.x and s.y == n_s.y):
      if (a == left):
        return (1 - p_e + (p_e/4))
      else: 
        return (p_e / 4)
    elif (s.x + 1 == n_s.x and s.y == n_s.y): 
      if (a == right):
        return (1 - p_e + (p_e/4))
      else: 
        return (p_e / 4)
    elif ( s.x  == n_s.x and s.y + 1 == n_s.y):
      if (a == up):
        return (1 - p_e + (p_e/4))
      else: 
        return (p_e / 4)
    elif (s.x  == n_s.x and s.y - 1 == n_s.y):
      if (a == down):
        return (1 - p_e + (p_e/4))
      else: 
        return (p_e / 4)

        
    # probability of staying still in non-corner/non-corridor space when moving towards
    # a wall/obstacle is 1 - Pe + Pe/4
    elif (s.x == n_s.x and s.y == n_s.y) and \
            ((a == left and ((s.x == 0 and 1 < s.y < 5 and s.y != 3) or (s.x == 3 and (s.y == 3 or s.y == 1)))) or
              (a == right and s.x == 4 and 0 < s.y < 5) or
              (a == up and 0 < s.x < 4 and s.y == 5) or
              (a == down and ((s.x == 3 and s.y == 0) or ((s.x == 1 or s.x == 2) and s.y == 4)))):
        return (1 - p_e + (p_e/4))

    # probability of staying still when moving away from a wall/obstacle is Pe - Pe/4
    elif (s.x == n_s.x and s.y == n_s.y) and \
            (((s.x == 0 and 1 < s.y < 5 and s.y != 3) or (s.x == 3 and (s.y == 3 or s.y == 1))) or
              (s.x == 4 and 0 < s.y < 5) or
              (0 < s.x < 4 and s.y == 5) or
              ((s.x == 3 and s.y == 0) or ((s.x == 1 or s.x == 2) and s.y == 4))):
        return (p_e / 4)

    # probability of staying still in a corner/corridor while moving towards
    # wall/obstacle is 1 - Pe + Pe/2
    elif (s.x == n_s.x and s.y ==  n_s.y) and \
            ((a == left and s.x == 0) or
              (a == right and ((s.x == 0 and (s.y == 1 or s.y == 3)) or s.x == 4)) or
              (a.move == up and (((s.x == 1 or s.x == 2) and (s.y == 0 or s.y == 2)) or s.y == 5)) or
              (a == down and (((s.x == 1 or s.x == 2) and s.y == 2) or s.y == 0))):
        return (1 - p_e + (p_e / 2))

    # probability of staying still in a corner while moving away from wall is Pe - Pe/2
    elif s.x ==  n_s.x and s.y ==  n_s.y and (s.x == 0 or s.x == 4 or s.y == 5 or s.y == 0):
        return (p_e / 2)
    
    # probability of transitioning to yellow blocks is zero so remains zero
    elif (n_s.y == 1 or n_s.y == 3) and (n_s.x == 1 or n_s.x == 2):
        return 0

    # all other motions(pretty much teleportation) are impossible so remain zero
    else:
        return 0




2b: creating the rewards function.  This is based on the chart given where there are two greater than 0 spots (one +1 and one +10).  There is an area of essentially high danger of -100.  The rest is all zero 

In [0]:
def get_reward(s):
  if (s.x == 2) and (s.y == 0): 
    return 10 #highest rewards spot
  elif (s.x == 2) and (s.y == 2):
    return 1 #second highest reward spot
  elif(s.x == 4):
    return -100 #danger zone 
  else: 
    return 0 #everywhere else is zero 

3a: I created the initial pi matrix using a function.  This way I can recreate it for both value and policy iteration without worrying if the array is getting overriden

In [0]:
def get_pi_initial():
  #everywhere essentially is left
  #left = 0 so i used the np zero fill function
  pi_initial = np.zeros((L,H))

  #there are four spots of obstacles though
  #there are marked with a 5 or obstacle 
  pi_initial[1,1] = obstacle 
  pi_initial[2,1] = obstacle
  pi_initial[1,3] = obstacle
  pi_initial[2,3] = obstacle
  return pi_initial

3b: function that displays the a nice chart of the policy.  This uses L = left, R = right, U = up, D = down, N = none (no movement), - = obstacle 

In [74]:
def display_policy(pi):
  #the grid world starts with 0,0 at the bottom left 
  #had to 'reverse' y to start at the bottom 
  for y in reversed(range(H)): 
    for x in range(L):
      #adding end =" " supresses the automatic newline with print
      if pi[x][y]==left:
        print("L ", end=" ")
      elif pi[x][y]==right:
        print("R ", end=" ")
      elif pi[x][y]==up:
        print("U ", end=" ")
      elif pi[x][y]==down:
        print("D ", end=" ")
      elif pi[x][y]==none: 
        print("N ", end=" ")
      elif pi[x][y]==obstacle:
        print("- ", end=" ")
    print("\n") #only newline when a new row is starting

#displaying the pi intial 
pi_initial = get_pi_initial()
display_policy(pi_initial)

L  L  L  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  



3c: Policy evaluation function and its two helpers.  am_i_obstacle(s) returns if s is at an obstacle or outof bounds. get_value(x,y,values,a) calculates the new value at state (x,y) given the action. policy evaluation uses these two functions to create a value matrix. 

In [0]:
def am_i_obstacle(s):
  if (s.x==1):
    if((s.y==3) or (s.y==1)): #obstacle
      return 1;
  elif (s.x==2):
    if((s.y==3) or (s.y==1)):#obstacle
      return 1;
  if ((s.x < 0) or (s.y < 0) or (s.x >= L) or (s.y >= H)):#out of bounds
    return 1; 
  return 0;  

def get_value(x, y, values, a):
  
  s = state(x, y) 

  #if we are an obstacle, the value is just 0 
  if am_i_obstacle(s)==1:
    return 0

  new_value = 0 
  
  #attempt each of the actions by ensuring that taking each of the actions 
  #does not result in going out of bounds or into a wall 
  go_left = state(x-1,y)
  if am_i_obstacle(go_left) == 0: 
    new_value += get_probability(s, a, go_left)*(get_reward(go_left)+gamma*values[x-1][y])
      
  go_right = state(x+1,y) 
  if am_i_obstacle(go_right) == 0:
    new_value+= get_probability(s, a, go_right)*(get_reward(go_right)+gamma*values[x+1][y])

  go_up = state(x,y+1) 
  if am_i_obstacle(go_up) == 0:
    new_value+= get_probability(s, a, go_up)*(get_reward(go_up)+gamma*values[x][y+1])

  go_down = state(x,y-1) 
  if am_i_obstacle(go_down) == 0:
    new_value+= get_probability(s, a, go_down)*(get_reward(go_down)+gamma*values[x][y-1])

  new_value += get_probability(s, a, s)*(get_reward(s)+gamma*values[x][y])

  return new_value

def policy_evaluation(values, pi):
  
  for x in range (L): 
    for y in range (H): 
      values[x][y]=get_value(x, y, values, pi[x][y])
  return values




3d: this does the bellman one step lookahead for the pi matrix by seeing which value (given all of the possible actions) is the highest

In [77]:
def policy_onestep_lookahead (values, pi):
  for x in range(L):
    for y in range(H): 

      if (pi[x][y] == obstacle):
        continue
      #for each action, calculate the value 
      left_value = get_value(x, y, values, left)
      right_value = get_value(x, y, values, right)
      up_value = get_value(x, y, values, up)
      down_value = get_value(x, y, values, down)
      none_value = get_value(x, y, values, none)

      #take the max value, and use the corresonding action
      pi[x][y] = np.argmax([left_value, right_value, up_value, down_value, none_value])

  return pi 
#testing one step with initials 
#all values are initially zero
values_initial = np.zeros((L,H))
#get the pi initial 
pi_initial = get_pi_initial()
#see what pi1 is after giving it pi0
pi_1 = policy_onestep_lookahead(values_initial, pi_initial)
display_policy(pi_1)
#print(pi_1)

L  L  L  N  L  

L  L  L  N  L  

L  -  -  N  L  

L  R  N  L  L  

L  -  -  N  L  

L  R  N  L  L  



3e: function for policy iteration.  This will repeat the policy evaluation and optimizing it with the bellman one step lookahead until there is no more changes in the value function (meaning we have converged)

In [114]:
def policy_iteration(): 
  #start with the initial policy pi0 given  
  pi = get_pi_initial() 
  #for pi0 all values are 0 
  values = np.zeros((L,H))

  while 1: 
    value_previous = np.linalg.norm(values)
    #calculate new values 
    values = policy_evaluation(values, pi)
    #get the new pi from optimizing 
    pi = policy_onestep_lookahead(values, pi)
    next_value = np.linalg.norm(values)
    #compare the next and previous value matrices to see if they different
    if np.abs(next_value-value_previous) < 1e-10:
      break #if they are different we are done 
      
  #display the final pi matrix found 
  display_policy(pi)
  
policy_iteration()


D  L  L  L  L  

D  L  L  L  L  

D  -  -  N  L  

D  L  L  D  L  

D  -  -  D  L  

R  R  N  L  L  



4a: Value iteration done with using a value_onestep(values) function to calculate the best value based on all actions for a single state.  

In [0]:
def value_onestep(values):
  for x in range(L): 
    for y in range(H): 
      #similar to before try every single action
        left_value = get_value(x, y, values, left)
        right_value = get_value(x, y, values, right)
        up_value = get_value(x, y, values, up)
        down_value = get_value(x, y, values, down)
        none_value = get_value(x, y, values, none)

      #then find the one with the highest value to optimize 
        values[x][y] = np.max([left_value, right_value, up_value, down_value, none_value])
  return values


In [95]:
def value_iteration(): 

  #start with an zero value matrix 
  values = np.zeros((L,H))

  while True: 
    #continually calculate the next value matrix 
    previous_value = np.linalg.norm(values)
    values = value_onestep(values)
    next_value = np.linalg.norm(values)
    #compare to see if we are done
    if np.abs(next_value-previous_value) < 1e-10: 
      break 
  #once done get the best pi matrix from the found value matrix 
  pi = get_pi_initial()
  pi = policy_onestep_lookahead(values, pi)
  #display the policy found from the pi
  display_policy(pi)

value_iteration()

D  D  D  D  L  

D  L  R  D  L  

D  -  -  D  L  

D  R  R  D  L  

D  -  -  D  L  

R  R  N  L  L  



This is the top module.  It runs policy and value iteration based on a probability error and a discount (aka gamma).  There are multiple examples of running top already below.  Please note that the pi initial will be the same for all cases and to change this, you have to change the get_pi_intial function. 

In [109]:
def top(new_p_e, new_gamma): 

  #reset the probability error and discount with the new values 
  global p_e
  p_e = new_p_e 
  global gamma
  gamma = new_gamma 

  #run initial 
  print("Running case with error probability %.2f and discount %.2f "% (p_e, gamma))
  print("Pi initial: ")
  pi0 = get_pi_initial()
  display_policy(pi0)

  #run policy iteration 
  print("Running Policy Iteration: ")
  policy_iteration()

  #run value iteration 
  print("Running Value Iteration: ")
  value_iteration() 

top(0.01, 0.9)

Running case with error probability 0.01 and discount 0.90 
Pi initial: 
L  L  L  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  

Running Policy Iteration: 
D  D  D  D  L  

D  L  R  D  L  

D  -  -  D  L  

D  R  R  D  L  

D  -  -  D  L  

R  R  N  L  L  

Running Value Iteration: 
D  D  D  D  L  

D  L  R  D  L  

D  -  -  D  L  

D  R  R  D  L  

D  -  -  D  L  

R  R  N  L  L  



In [110]:
top(0.01, 0.1)

Running case with error probability 0.01 and discount 0.10 
Pi initial: 
L  L  L  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  

Running Policy Iteration: 
D  L  L  N  L  

D  L  L  N  L  

D  -  -  N  L  

R  R  N  L  L  

D  -  -  D  L  

R  R  N  L  L  

Running Value Iteration: 
D  L  L  N  L  

D  L  L  N  L  

D  -  -  N  L  

R  R  N  L  L  

D  -  -  D  L  

R  R  N  L  L  



In [111]:
top(0.3, 0.1)

Running case with error probability 0.30 and discount 0.10 
Pi initial: 
L  L  L  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  

Running Policy Iteration: 
D  L  L  N  L  

D  L  L  N  L  

D  -  -  N  L  

R  R  N  N  L  

D  -  -  N  L  

R  R  N  L  L  

Running Value Iteration: 
D  L  L  N  L  

D  L  L  N  L  

D  -  -  N  L  

R  R  N  N  L  

D  -  -  N  L  

R  R  N  L  L  



In [112]:
top(0.1, 0.3)

Running case with error probability 0.10 and discount 0.30 
Pi initial: 
L  L  L  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  

Running Policy Iteration: 
D  L  L  N  L  

D  L  L  N  L  

D  -  -  N  L  

R  R  N  N  L  

D  -  -  D  L  

R  R  N  L  L  

Running Value Iteration: 
D  L  L  N  L  

D  L  L  N  L  

D  -  -  N  L  

R  R  N  N  L  

D  -  -  D  L  

R  R  N  L  L  



In [113]:
top(0.4, 0.9)

Running case with error probability 0.40 and discount 0.90 
Pi initial: 
L  L  L  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  

Running Policy Iteration: 
D  L  L  L  L  

D  L  L  L  L  

D  -  -  N  L  

D  L  L  D  L  

D  -  -  D  L  

R  R  N  L  L  

Running Value Iteration: 
D  L  L  L  L  

D  L  L  L  L  

D  -  -  N  L  

D  L  L  D  L  

D  -  -  D  L  

R  R  N  L  L  

